#### Créer une video à partir de la séquence de frames sous img1

In [ ]:
import cv2
import os
import configparser
import glob

# Fonction pour traiter une séquence individuelle
def process_sequence(path_seq, video_output):
    config = configparser.ConfigParser()
    config.read(os.path.join(path_seq, 'seqinfo.ini'))

    sequence_info = config['Sequence']

    im_dir = sequence_info['imDir']
    frame_rate = int(sequence_info['frameRate'])
    seq_length = int(sequence_info['seqLength'])
    im_width = int(sequence_info['imWidth'])
    im_height = int(sequence_info['imHeight'])
    im_ext = sequence_info['imExt']

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(video_output, fourcc, frame_rate, (im_width, im_height))

    image_files = sorted(os.listdir(path_seq+'/'+im_dir))

    frames_processed = 0

    for image_file in image_files:
        if frames_processed >= seq_length:
            break

        if image_file.endswith(im_ext):
            image_path = os.path.join(path_seq+'/'+im_dir, image_file)
            img = cv2.imread(image_path)
            if img is not None:
                video_writer.write(img)
                frames_processed += 1
            else:
                print(f"Erreur lors de la lecture de l'image : {image_path}")

    video_writer.release()


# Chemin du répertoire contenant les séquences MOT17
mot17_path = '/home/jovyan/iadatasets/MOT/MOT17/train/'
output_dir = '/home/jovyan/Desktop/mot_videos/'

# Liste des séquences FRCNN
sequences = glob.glob(os.path.join(mot17_path, "*-FRCNN"))

# Traiter chaque séquence FRCNN et sauvegarder la vidéo dans le répertoire de sortie
for seq in sequences:
    seq_name = os.path.basename(seq)
    video_output = os.path.join(output_dir, f"{seq_name}.mp4")
    process_sequence(seq, video_output)


#### Afficher les informations de la vidéo (faut surtout vérifier le nbre de frames)

In [ ]:
import cv2

# Ouvrir la vidéo
video_output='/home/jovyan/Desktop/MOT_VIDEOS/MOT17-09-FRCNN.mp4'
video = cv2.VideoCapture(video_output)

# Récupérer les informations de la vidéo
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(video.get(cv2.CAP_PROP_FPS))
video_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = frame_count / frame_rate

# Afficher les informations de la vidéo
print(f"Nombre de frames: {frame_count}")
print(f"Frame rate: {frame_rate} FPS")
print(f"Largeur: {video_width}")
print(f"Hauteur: {video_height}")
print(f"Durée: {duration:.2f} secondes")

# Libérer les ressources
video.release()


#### Créer un file det.txt à partir des détections de efficientdet.pth appliqué à une vidéo

In [36]:
def save_detections_to_file(det_results, output_file):
    with open(output_file, 'w') as f:
        for frame_id, detections in det_results:
            for det in detections:
                # Les coordonnées de la boîte englobante
                bb_left = det[0]
                bb_top = det[1]
                bb_width = det[2] - det[0]
                bb_height = det[3] - det[1]
                # La confiance de la détection
                conf = det[4]
                # L'id de l'objet est indéterminé; -1
                obj_id = -1
                # Les coordonnées 3D sont indéterminées, donc on met -1
                x = y = z = -1
                # Écrire la détection dans le fichier
                f.write(f"{frame_id},-1,{bb_left},{bb_top},{bb_width},{bb_height},{conf},{x},{y},{z}\n")
             #   print(bb_left,bb_top,bb_width,bb_height,conf)


In [1]:
import cv2
from backbone import EfficientDetBackbone
from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess, preprocess_video_frame,preprocess_video
import torch
import time
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np


compound_coef = 2
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]
use_cuda = True
use_float16 = False
gpu = 0
threshold = 0.2
nms_threshold = 0.1
params = {
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225],
    'anchors_scales': '[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]',
    'anchors_ratios': '[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]',
        }
video_path='/home/jovyan/Desktop/MOT_VIDEOS/MOT17-09-FRCNN.mp4'


# Charger le mod  le
model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=90)
state_dict = torch.load('/home/jovyan/Desktop/Pytorch/weights/efficientdet-d2.pth')
model.load_state_dict(state_dict, strict=False)

for param in model.parameters():
    param.requires_grad = False
model.eval()

# Utiliser le GPU si n  cessaire
if use_cuda:
    model.cuda(gpu)
    if use_float16:
        model.half()

# Initialiser les objets pour la post-processing
regressBoxes = BBoxTransform()
clipBoxes = ClipBoxes()

cap = cv2.VideoCapture(video_path)
frame_id = 1
det_results = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break


     # Pr  traiter l'image
    ori_img, framed_imgs, framed_meta = preprocess_video(frame, max_size=input_sizes[compound_coef],
                                                         mean=params['mean'], std=params['std'])
   # print('ori_img.shape',ori_img.shape)
  #  print('framed_imgs',framed_imgs)
  #  print('framed_meta',framed_meta)

    if use_cuda:
        x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
    else:
        x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)

    x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

    # Passer l'image à travers EfficientDet
    features, regression, classification, anchors = model(x)
    target_size = features[3].shape[-2:] 
    print('target_size',target_size)
    # On fait un interpolate (upsampling) pour chaque feature map pour atteindre la taille voulue, puis on les concatène
    resized_features = [F.interpolate(f, size=target_size, mode='bilinear', align_corners=False) for f in features]
    combined_feature = torch.cat(resized_features, dim=1)
    print('combined_feature initial',combined_feature)
   



    # Post-traiter les predictions
    preds = postprocess(x,
                        anchors, regression, classification,
                        regressBoxes, clipBoxes,
                        threshold, nms_threshold)

    if preds:
        # Inverser la transformation appliquée à l'image
        preds = invert_affine(framed_meta, preds)[0]
        #print('preds',preds)
        # Récupérer les détections
        scores = preds['scores']
        class_ids = preds['class_ids']
        rois = preds['rois']
        print('rois',rois)
        print('scores',scores)
        #la liste des détections pour ce frame
        
        det_results.append((frame_id, [[int(x1), int(y1), int(x2), int(y2), score] for (x1, y1, x2, y2), score in zip(rois.tolist(), scores.tolist())]))

    print('cropped_features',cropped_features)

    print('cropped_features',cropped_features.shape)
    frame_id += 1
    break
    
    
    
# Enregistrer les détections dans un fichier
save_detections_to_file(det_results, 'output.txt')


target_size torch.Size([12, 12])
combined_feature initial tensor([[[[ 2.7530e-01, -4.5979e-01,  3.7629e-01,  ...,  4.5612e-01,
            7.2212e-01,  4.7469e-01],
          [-3.6784e-01,  4.5565e-02,  2.1484e-01,  ...,  4.1473e-01,
            1.0398e+00,  3.4476e-02],
          [ 1.4328e+00,  6.8675e-01, -5.0410e-01,  ...,  9.9381e-01,
            2.5067e-01,  2.2186e-01],
          ...,
          [-3.0292e-02,  1.0623e+00, -9.7127e-01,  ..., -1.4080e-01,
           -1.2552e+00, -1.5818e-01],
          [ 4.7368e-02, -6.4292e-01, -5.6723e-01,  ...,  2.8653e-01,
           -1.1101e+00,  6.4429e-01],
          [ 6.8558e-01, -3.0504e-02, -5.1785e-01,  ..., -3.3316e-01,
           -1.7179e+00, -4.5952e-01]],

         [[-1.7163e-01, -3.3242e-02,  3.3638e-01,  ..., -6.5499e-01,
           -4.8732e-01,  6.4229e-01],
          [ 2.3364e-01,  6.8937e-01,  2.0814e-01,  ..., -1.5072e+00,
           -1.7049e+00,  6.4442e-01],
          [-6.0212e-01, -8.1162e-01, -1.1984e+00,  ..., -1.0669e+00,


NameError: name 'cropped_features' is not defined

In [46]:
from torchvision.ops import roi_align

class RoIAlign(nn.Module):
    def __init__(self, crop_height, crop_width, transform_fpcoor=True, sampling_ratio=-1):
        super(RoIAlign, self).__init__()

        self.crop_height = crop_height
        self.crop_width = crop_width
        self.transform_fpcoor = transform_fpcoor
        self.sampling_ratio = sampling_ratio

    def forward(self, featuremap, boxes, box_ind):
        boxes = boxes.detach().contiguous()  # detacher du calcul du gradient,
        box_ind = box_ind.detach()
        return roi_align(featuremap, boxes, (self.crop_height, self.crop_width), sampling_ratio=self.sampling_ratio)


### feature pkl file 

In [ ]:
import os
import cv2
import torch
import pickle
import numpy as np
from backbone import EfficientDetBackbone
import torch.nn.functional as F

# Paramètres
compound_coef = 1
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]
use_cuda = True
use_float16 = False
gpu = 0
params = {
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225],
}
image_dir = '/home/jovyan/iadatasets/MOT/MOT17/train/MOT17-02-FRCNN/img1'
output_dir = '/home/jovyan/Desktop/feature/'
os.makedirs(output_dir, exist_ok=True)

# Charger le modèle
model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=90)
state_dict = torch.load('/home/jovyan/Desktop/Pytorch/weights/efficientdet-d1.pth')
model.load_state_dict(state_dict, strict=False)

for param in model.parameters():
    param.requires_grad = False
model.eval()

if use_cuda:
    model.cuda(gpu)
    if use_float16:
        model.half()

# Boucle sur toutes les images
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'): 
        # Charger l'image
        image = cv2.imread(os.path.join(image_dir, filename))
        if image is None:
            continue

        # Taille d'entrée pour EfficientDet-d1
        image_size = input_sizes[compound_coef]  # i.e., 640

        # Resize
        image = cv2.resize(image, (image_size, image_size))
        image = image.astype(np.float32) / 255
        image -= params['mean']
        image /= params['std']
        image = np.transpose(image, (2, 0, 1))
        image = np.expand_dims(image, axis=0)

        # Passer l'image à travers EfficientDet
        with torch.no_grad():
            if use_cuda:
                image = torch.from_numpy(image).cuda().float()
            else:
                image = torch.from_numpy(image).float()
            features, _, _, _ = model(image)
        #    print('features[0] shape',features[0].shape)
        #    print('features[0] shape',features[1].shape)
        #    print('features[0] shape',features[2].shape)
        #    print('features[0] shape',features[3].shape)
        #    print('features[0] shape',features[4].shape)
            
            # On détermine la taille cible comme la taille de la plus grande feature map
            target_size = features[0].shape[-2:]  # Assuming features[0] is the largest one

            # On fait un interpolate (upsampling) pour chaque feature map pour atteindre la taille cible, puis on les concatène
            resized_features = [F.interpolate(f, size=target_size, mode='bilinear', align_corners=False) for f in features]
            combined_feature = torch.cat(resized_features, dim=1)
         #   print('combined_feature type)', combined_feature.shape)
        # Sauvegarder les caractéristiques dans un fichier .pkl
        feature_file = os.path.join(output_dir, filename.split('.')[0]+ '.pkl')
        with open(feature_file, 'wb') as f:
            pickle.dump(combined_feature.cpu().numpy(), f)
